In [38]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed

In [18]:
data = pd.read_csv('temperature_data.csv')
data.head()

,city,timestamp,temperature,season
0,New York,2010-01-01,13.694016,winter
1,New York,2010-01-02,1.703563,winter
2,New York,2010-01-03,2.770039,winter
3,New York,2010-01-04,1.067388,winter
4,New York,2010-01-05,-5.455941,winter


In [ ]:
# Скользящее среднее за последние 30 дней (или меньше, если данных еще нет)
data['moving_average'] = data['temperature'].rolling(window=30, min_periods=1).mean()

seasons = data['season'].unique()
towns = data['city'].unique()

df = pd.DataFrame(columns=['city', 'season', 'average', 'standard_deviation'])

for town in towns:
    for season in seasons:
        ts_mean = data[(data['season'] == season) & (data['city'] == town)]['temperature'].mean()
        ts_std = data[(data['season'] == season) & (data['city'] == town)]['temperature'].std()
        df = pd.concat([df, pd.DataFrame({'city': [town], 'season': [season], 'average': [ts_mean], 'standard_deviation': [ts_std]})])

def is_anomaly(row):
    av = df['average'][(df['city'] == row['city']) & (df['season'] == row['season'])]
    st = df['standard_deviation'][(df['city'] == row['city']) & (df['season'] == row['season'])]
    return abs(row['temperature'] - av) > 2 * st




/var/folders/rw/smqkxbj14bj23k5pvwqk3csh0000gn/T/ipykernel_17029/2891374832.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame({'city': [town], 'season': [season], 'average': [ts_mean], 'standard_deviation': [ts_std]})])
UsageError: Line magic function `%%time` not found.


In [42]:
%%time
data['is_anomaly'] = data.apply(is_anomaly, axis=1)

CPU times: user 14.7 s, sys: 396 ms, total: 15.1 s
Wall time: 14.9 s


In [ ]:
%%time
n_workers = 8
chunks = np.array_split(data, n_workers)

def process_chunk(df_chunk):
    return df_chunk.apply(is_anomaly, axis=1)

parts = Parallel(n_jobs=n_workers)(delayed(process_chunk)(ch) for ch in chunks)

data["is_anomaly"] = pd.concat(parts).sort_index()

/Users/vladimirovdma/Library/Python/3.9/lib/python/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


CPU times: user 11.4 ms, sys: 3.33 ms, total: 14.7 ms
Wall time: 1.97 s


In [45]:
data.head()
df.to_csv('temperature_data_aggregates.csv')